In [24]:
import os
import random
import itertools
import re

# 基本的なライブラリ
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# 描画ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn_analyzer import CustomPairPlot
import graphviz
import pydotplus
from IPython.display import Image
from IPython.display import HTML
from six import StringIO
from ipywidgets import interact, FloatSlider
from dtreeviz.trees import *

# データセット分割
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit

# 補完
from sklearn.experimental import (
    enable_iterative_imputer,
)  # IterativeImputerをimportするために必要
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# エンコード
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder

# サンプリング
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# 特徴量選択
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import (
    GenericUnivariateSelect,
    f_classif,
    mutual_info_classif,
    chi2,
)
from boruta import BorutaPy
from libraries.mrmr import mrmr

# 学習中
import optuna
from tqdm import tqdm
from sklearn.model_selection import learning_curve, cross_validate, cross_val_score

# 評価指標
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# models
from lightgbm import LGBMClassifier

# XAI
import shap

import warnings


# config python file
import config

SEED = config.SEED

from functions import *

fix_seed(SEED)


# 最大表示列数の指定（ここでは50列を指定）N
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

%matplotlib inline

# データ読み込み

In [27]:
X_dict = pickle_load(
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/X_dict.pickle"
)
y_dict = pickle_load(
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/y_dict.pickle"
)

In [29]:
X_train = X_dict["genes"]["mrmr"]["train"]["X15"]
y_train = y_dict["genes"]["mrmr"]["train"]["y15"]
X_val = X_dict["genes"]["mrmr"]["val"]["X15"]
y_val = y_dict["genes"]["mrmr"]["val"]["y15"]

assert X_train.shape[0] == y_train.shape[0], "train size is incorrect"
assert X_val.shape[0] == y_val.shape[0], "test size is incorrect"
assert X_train.shape[1] == X_val.shape[1], "feature size is incorrect"

# accuracyの表示
print("----------" * 10)
print("使用特徴量：", X_train.columns)
print("学習サンプルサイズ：", X_train.shape)
display("ラベル比率：", y_train.value_counts())

----------------------------------------------------------------------------------------------------
使用特徴量： Index(['RACGAP1', 'TBC1D24', 'ARTN', 'LIN7A', 'IGFBP5', 'ANGPT2', 'HAMP',
       'UTP23', 'RBBP8', 'STAT5B', 'C9orf95', 'FGF13', 'RNU11', 'ENC1', 'AK3',
       'NEIL2', 'CS', 'S100P', 'FGD3', 'IL20RA', 'WDR67', 'C2orf74',
       'HIST1H2BF', 'ADCK2', 'PREX1', 'INTS10', 'ZIC2', 'DPM3', 'SUSD3',
       'PPIL3', 'RND1', 'GSTM2', 'KIAA1967', 'COX7B2', 'SPATA18'],
      dtype='object')
学習サンプルサイズ： (855, 35)


'ラベル比率：'

1    458
0    397
Name: OS_15years, dtype: int64

# 最適化(all data)

分類器を学習させ、パラメータのチューニングを行い、高い予測精度を目指す。

## Optuna

### Random Forest

In [30]:
def objective(trial):
    # ランダムフォレストのパラメータチューニング
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "max_depth": trial.suggest_int("max_depth", 2, 50, log=True),
        "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 2, 100),
        "class_weight": trial.suggest_categorical(
            "class_wight", ["balanced", "balanced_subsample"]
        ),
        "random_state": SEED,
    }

    clf = RandomForestClassifier(**params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(
        clf, X_train, y_train, n_jobs=-1, cv=10, scoring=make_scorer(f1_score)
    )
    return score.mean()


study = optuna.create_study(
    direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED)
)
study.optimize(objective, n_trials=100)

In [31]:
study.best_trial.params

{'n_estimators': 70,
 'max_depth': 42,
 'max_leaf_nodes': 62,
 'class_wight': 'balanced'}

### LogisticRegression

In [66]:
def objective(trial):
    # ランダムフォレストのパラメータチューニング
    params = {
        "penalty": trial.suggest_categorical("penalty", ["l1", "l2"]),
        "C": trial.suggest_float("C", 1e-5, 1e5),
        "max_iter": 1000,
        "class_weight": "balanced",
        "random_state": SEED,
    }

    clf = LogisticRegression(**params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(
        clf, X_train, y_train, n_jobs=-1, cv=10, scoring=make_scorer(f1_score)
    )
    return score.mean()


study = optuna.create_study(
    direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED)
)
study.optimize(objective, n_trials=10)

Trial 0 failed, because the objective function returned nan.
Trial 1 failed, because the objective function returned nan.
Trial 4 failed, because the objective function returned nan.
Trial 5 failed, because the objective function returned nan.
Trial 6 failed, because the objective function returned nan.
Trial 7 failed, because the objective function returned nan.


In [67]:
study.best_trial.params, study.best_trial.values

{'penalty': 'l2', 'C': 13670.65897712823}

### SVC

In [68]:
def objective(trial):
    # ランダムフォレストのパラメータチューニング
    params = {
        "C": trial.suggest_float("C", 1e-5, 1e5),
        "kernel": trial.suggest_categorical(
            "kernel",
            [
                "linear",
                "poly",
                "rbf",
                "sigmoid",
            ],
        ),
        "gamma": trial.suggest_categorical("gamma", ["scale", "auto"]),
        "shrinking": trial.suggest_categorical("shrinking", [True, False]),
        "decision_function_shape": trial.suggest_categorical(
            "decision_function_shape", ["ovo", "ovr"]
        ),
        "class_weight": "balanced",
        "random_state": SEED,
    }

    clf = SVC(**params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(
        clf, X_train, y_train, n_jobs=-1, cv=10, scoring=make_scorer(f1_score)
    )
    return score.mean()


study = optuna.create_study(
    direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED)
)
study.optimize(objective, n_trials=100)

In [69]:
study.best_trial.params, study.best_trial.values

{'C': 31653.691881778464,
 'kernel': 'linear',
 'gamma': 'scale',
 'shrinking': False,
 'decision_function_shape': 'ovr'}

### lightGBM

In [70]:
def objective(trial):
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "max_bin": trial.suggest_int("max_bin", 100, 300),
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "class_weight": "balanced",
        "random_state": SEED,
    }
    clf = LGBMClassifier(boosting_type="gbdt", **params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=10)
    return score.mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[LightGBM] [Warning] lambda_l1 is set=0.1594461381461486, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1594461381461486
[LightGBM] [Warning] lambda_l2 is set=0.14874108376646833, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.14874108376646833
[LightGBM] [Warning] lambda_l1 is set=8.290460059587413e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.290460059587413e-06
[LightGBM] [Warning] lambda_l2 is set=7.15917297984776e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.15917297984776e-08
[LightGBM] [Warning] lambda_l1 is set=1.727528199465488, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.727528199465488
[LightGBM] [Warning] lambda_l2 is set=0.007065839141742113, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.007065839141742113
[LightGBM] [Warning] lambda_l1 is set=1.185569229064907, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.185569229064907
[LightGBM] [Warning] lambda_l2 is set=2.27151549092021

In [72]:
study.best_trial.params, study.best_trial.values

({'lambda_l1': 2.3140471298414025,
  'lambda_l2': 2.8001110203753297,
  'max_bin': 283,
  'num_leaves': 20,
  'learning_rate': 0.01727053317047955,
  'n_estimators': 351},
 [0.7099863201094392])

# 最適化(subtypes)

分類器を学習させ、パラメータのチューニングを行い、高い予測精度を目指す。

In [73]:
dict_subtype_values = dict()
dict_subtype_params = dict()

## RandomForest

In [84]:
def objective(trial):
    warnings.simplefilter("ignore")
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "max_depth": trial.suggest_int("max_depth", 2, 50, log=True),
        "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 2, 100),
        "class_weight": trial.suggest_categorical(
            "class_weight", ["balanced", "balanced_subsample"]
        ),
        "random_state": SEED,
    }

    clf = RandomForestClassifier(**params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(clf, X_train_subtype, y_train_subtype, n_jobs=-1, cv=10)
    return score.mean()

In [85]:
optuna.logging.disable_default_handler()
warnings.simplefilter("ignore")
subtypes = [
    "claudin-low",
    "LumA",
    "LumB",
    "Her2",
    "Normal",
    "Basal",
]
dict_subtype_values["rf"] = dict()
dict_subtype_params["rf"] = dict()
for year in range(15, 16, 5):  # 予後年数毎のループ
    for subtype in tqdm(subtypes):
        dict_subtype_values["rf"][subtype] = dict()
        dict_subtype_params["rf"][subtype] = dict()
        prognosis_Xlabel = "X{0:0=2}_{1}".format(year, subtype)
        prognosis_ylabel = "y{0:0=2}_{1}".format(year, subtype)
        X_train_subtype = X_dict["genes"]["claudin_subtype"]["mrmr"]["train"][
            prognosis_Xlabel
        ]
        y_train_subtype = y_dict["genes"]["claudin_subtype"]["mrmr"]["train"][
            prognosis_ylabel
        ]
        assert (
            X_train_subtype.shape[0] == y_train_subtype.shape[0]
        ), "train size is incorrect"

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=100)
        dict_subtype_values["rf"][subtype][year] = study.best_trial.values
        dict_subtype_params["rf"][subtype][year] = study.best_trial.params

100%|█████████████████████████████████████████████████████████████████████████████████████| 6/6 [09:13<00:00, 92.23s/it]


## LogitsticRegression

In [77]:
def objective(trial):
    warnings.simplefilter("ignore")
    params = {
        "penalty": trial.suggest_categorical("penalty", ["l1", "l2"]),
        "C": trial.suggest_float("C", 1e-5, 1e5),
        "max_iter": 1000,
        "class_weight": "balanced",
        "random_state": SEED,
    }

    clf = LogisticRegression(**params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(clf, X_train_subtype, y_train_subtype, n_jobs=-1, cv=10)
    return score.mean()

In [78]:
optuna.logging.disable_default_handler()
warnings.simplefilter("ignore")
subtypes = [
    "claudin-low",
    "LumA",
    "LumB",
    "Her2",
    "Normal",
    "Basal",
]
dict_subtype_values["lr"] = dict()
dict_subtype_params["lr"] = dict()
for year in range(15, 16, 5):  # 予後年数毎のループ
    print("====={0:0=2}".format(year) * 10)
    for subtype in subtypes:
        print("----------" * 10)
        print(subtype)
        dict_subtype_values["lr"][subtype] = dict()
        dict_subtype_params["lr"][subtype] = dict()
        prognosis_Xlabel = "X{0:0=2}_{1}".format(year, subtype)
        prognosis_ylabel = "y{0:0=2}_{1}".format(year, subtype)
        X_train_subtype = X_dict["genes"]["claudin_subtype"]["mrmr"]["train"][
            prognosis_Xlabel
        ]
        y_train_subtype = y_dict["genes"]["claudin_subtype"]["mrmr"]["train"][
            prognosis_ylabel
        ]
        assert (
            X_train_subtype.shape[0] == y_train_subtype.shape[0]
        ), "train size is incorrect"

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=100)
        dict_subtype_values["lr"][subtype][year] = study.best_trial.values
        dict_subtype_params["lr"][subtype][year] = study.best_trial.params

Trial 0 failed, because the objective function returned nan.
Trial 2 failed, because the objective function returned nan.
Trial 3 failed, because the objective function returned nan.


=====15=====15=====15=====15=====15=====15=====15=====15=====15=====15
----------------------------------------------------------------------------------------------------
claudin-low


Trial 6 failed, because the objective function returned nan.
Trial 10 failed, because the objective function returned nan.
Trial 15 failed, because the objective function returned nan.
Trial 16 failed, because the objective function returned nan.
Trial 17 failed, because the objective function returned nan.
Trial 18 failed, because the objective function returned nan.
Trial 19 failed, because the objective function returned nan.
Trial 20 failed, because the objective function returned nan.
Trial 21 failed, because the objective function returned nan.
Trial 22 failed, because the objective function returned nan.
Trial 23 failed, because the objective function returned nan.
Trial 24 failed, because the objective function returned nan.
Trial 25 failed, because the objective function returned nan.
Trial 26 failed, because the objective function returned nan.
Trial 27 failed, because the objective function returned nan.
Trial 28 failed, because the objective function returned nan.
Trial 29 

----------------------------------------------------------------------------------------------------
LumA


Trial 4 failed, because the objective function returned nan.
Trial 5 failed, because the objective function returned nan.
Trial 6 failed, because the objective function returned nan.
Trial 8 failed, because the objective function returned nan.
Trial 10 failed, because the objective function returned nan.
Trial 11 failed, because the objective function returned nan.
Trial 12 failed, because the objective function returned nan.
Trial 13 failed, because the objective function returned nan.
Trial 14 failed, because the objective function returned nan.
Trial 18 failed, because the objective function returned nan.
Trial 19 failed, because the objective function returned nan.
Trial 20 failed, because the objective function returned nan.
Trial 22 failed, because the objective function returned nan.
Trial 23 failed, because the objective function returned nan.
Trial 24 failed, because the objective function returned nan.
Trial 25 failed, because the objective function returned nan.
Trial 26 fai

...

Trial 3 failed, because the objective function returned nan.
Trial 10 failed, because the objective function returned nan.
Trial 11 failed, because the objective function returned nan.
Trial 12 failed, because the objective function returned nan.
Trial 13 failed, because the objective function returned nan.
Trial 14 failed, because the objective function returned nan.
Trial 17 failed, because the objective function returned nan.
Trial 18 failed, because the objective function returned nan.
Trial 19 failed, because the objective function returned nan.
Trial 20 failed, because the objective function returned nan.
Trial 21 failed, because the objective function returned nan.
Trial 22 failed, because the objective function returned nan.
Trial 23 failed, because the objective function returned nan.
Trial 24 failed, because the objective function returned nan.
Trial 25 failed, because the objective function returned nan.
Trial 26 failed, because the objective function returned nan.
Trial 27 

----------------------------------------------------------------------------------------------------
Normal


Trial 3 failed, because the objective function returned nan.
Trial 8 failed, because the objective function returned nan.
Trial 9 failed, because the objective function returned nan.
Trial 11 failed, because the objective function returned nan.
Trial 13 failed, because the objective function returned nan.
Trial 14 failed, because the objective function returned nan.
Trial 15 failed, because the objective function returned nan.
Trial 16 failed, because the objective function returned nan.
Trial 17 failed, because the objective function returned nan.
Trial 19 failed, because the objective function returned nan.
Trial 21 failed, because the objective function returned nan.
Trial 22 failed, because the objective function returned nan.
Trial 23 failed, because the objective function returned nan.
Trial 24 failed, because the objective function returned nan.
Trial 25 failed, because the objective function returned nan.
Trial 26 failed, because the objective function returned nan.
Trial 27 fa

----------------------------------------------------------------------------------------------------
Basal


Trial 7 failed, because the objective function returned nan.
Trial 8 failed, because the objective function returned nan.
Trial 9 failed, because the objective function returned nan.
Trial 10 failed, because the objective function returned nan.
Trial 13 failed, because the objective function returned nan.
Trial 18 failed, because the objective function returned nan.
Trial 20 failed, because the objective function returned nan.
Trial 21 failed, because the objective function returned nan.
Trial 22 failed, because the objective function returned nan.
Trial 23 failed, because the objective function returned nan.
Trial 24 failed, because the objective function returned nan.
Trial 25 failed, because the objective function returned nan.
Trial 26 failed, because the objective function returned nan.
Trial 27 failed, because the objective function returned nan.
Trial 28 failed, because the objective function returned nan.
Trial 29 failed, because the objective function returned nan.
Trial 30 fa

### lightGBM

In [79]:
def objective(trial):
    warnings.simplefilter("ignore")
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "max_bin": trial.suggest_int("max_bin", 100, 300),
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "class_weight": "balanced",
        "random_state": SEED,
    }
    clf = LGBMClassifier(boosting_type="gbdt", **params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(clf, X_train_subtype, y_train_subtype, n_jobs=-1, cv=10)
    return score.mean()

In [80]:
optuna.logging.disable_default_handler()
warnings.simplefilter("ignore")
subtypes = [
    "claudin-low",
    "LumA",
    "LumB",
    "Her2",
    "Normal",
    "Basal",
]
dict_subtype_values["lgb"] = dict()
dict_subtype_params["lgb"] = dict()
for year in range(15, 16, 5):  # 予後年数毎のループ
    print("====={0:0=2}".format(year) * 10)
    for subtype in tqdm(subtypes):
        print("----------" * 10)
        print(subtype)
        dict_subtype_values["lgb"][subtype] = dict()
        dict_subtype_params["lgb"][subtype] = dict()
        prognosis_Xlabel = "X{0:0=2}_{1}".format(year, subtype)
        prognosis_ylabel = "y{0:0=2}_{1}".format(year, subtype)
        X_train_subtype = X_dict["genes"]["claudin_subtype"]["mrmr"]["train"][
            prognosis_Xlabel
        ]
        y_train_subtype = y_dict["genes"]["claudin_subtype"]["mrmr"]["train"][
            prognosis_ylabel
        ]
        assert (
            X_train_subtype.shape[0] == y_train_subtype.shape[0]
        ), "train size is incorrect"

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=100)
        dict_subtype_values["lgb"][subtype][year] = study.best_trial.values
        dict_subtype_params["lgb"][subtype][year] = study.best_trial.params

=====15=====15=====15=====15=====15=====15=====15=====15=====15=====15
----------------------------------------------------------------------------------------------------
claudin-low
[LightGBM] [Warning] lambda_l1 is set=0.008649480152804118, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.008649480152804118
[LightGBM] [Warning] lambda_l2 is set=6.626429774459896e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.626429774459896e-08
[LightGBM] [Warning] lambda_l1 is set=0.4968993349439986, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.4968993349439986
[LightGBM] [Warning] lambda_l2 is set=1.3091083109613483e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.3091083109613483e-06
[LightGBM] [Warning] lambda_l1 is set=7.657057579652043e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.657057579652043e-08
[LightGBM] [Warning] lambda_l2 is set=9.007120341106653e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.00712

In [86]:
make_dir(config.INTERIM_PICKLE_TUNING_PROGNOSIS_CROSS_DIR)
pickle_dump(
    dict_subtype_values,
    config.INTERIM_PICKLE_TUNING_PROGNOSIS_CROSS_DIR + "/subtype_values.pickle",
)
pickle_dump(
    dict_subtype_params,
    config.INTERIM_PICKLE_TUNING_PROGNOSIS_CROSS_DIR + "/subtype_params.pickle",
)

In [87]:
dict_subtype_values

{'rf': {'claudin-low': {15: [0.8861111111111111]},
  'LumA': {15: [0.7667989417989418]},
  'LumB': {15: [0.7809523809523808]},
  'Her2': {15: [0.808974358974359]},
  'Normal': {15: [0.9133333333333333]},
  'Basal': {15: [0.8300000000000001]}},
 'lr': {'claudin-low': {15: [0.8763888888888889]},
  'LumA': {15: [0.748015873015873]},
  'LumB': {15: [0.780952380952381]},
  'Her2': {15: [0.7769230769230769]},
  'Normal': {15: [0.9466666666666667]},
  'Basal': {15: [0.8727272727272727]}},
 'lgb': {'claudin-low': {15: [0.8319444444444445]},
  'LumA': {15: [0.8064814814814815]},
  'LumB': {15: [0.780952380952381]},
  'Her2': {15: [0.7756410256410257]},
  'Normal': {15: [0.9066666666666668]},
  'Basal': {15: [0.8036363636363637]}}}

In [88]:
dict_subtype_params

{'rf': {'claudin-low': {15: {'n_estimators': 478,
    'max_depth': 19,
    'max_leaf_nodes': 9,
    'class_weight': 'balanced_subsample'}},
  'LumA': {15: {'n_estimators': 882,
    'max_depth': 20,
    'max_leaf_nodes': 35,
    'class_weight': 'balanced'}},
  'LumB': {15: {'n_estimators': 97,
    'max_depth': 12,
    'max_leaf_nodes': 81,
    'class_weight': 'balanced_subsample'}},
  'Her2': {15: {'n_estimators': 608,
    'max_depth': 3,
    'max_leaf_nodes': 89,
    'class_weight': 'balanced'}},
  'Normal': {15: {'n_estimators': 91,
    'max_depth': 7,
    'max_leaf_nodes': 23,
    'class_weight': 'balanced'}},
  'Basal': {15: {'n_estimators': 517,
    'max_depth': 4,
    'max_leaf_nodes': 43,
    'class_weight': 'balanced'}}},
 'lr': {'claudin-low': {15: {'penalty': 'l2', 'C': 36156.402308163364}},
  'LumA': {15: {'penalty': 'l2', 'C': 9654.664188336264}},
  'LumB': {15: {'penalty': 'l2', 'C': 51528.43835221508}},
  'Her2': {15: {'penalty': 'l2', 'C': 5076.262956771448}},
  'Normal':